## Aula 04 - K-Nearest Neighbors

---

Vamos iniciar carregando os dados, embaralhando as amostras, e separando 100 amostras para teste e 50 amostras para treino,

In [ ]:
import numpy as np
import pandas as pd

df_dataset = pd.read_csv('iris.csv', sep=',', index_col=None)

In [ ]:
df_dataset.head()

In [ ]:
from sklearn.utils import shuffle
df_dataset = shuffle(df_dataset, random_state=42)

In [ ]:
df_dataset.head()

In [ ]:
treino = df_dataset[:100].values
teste = df_dataset[100:].values

print(treino.shape)
print(teste.shape)

---

Em seguida, vamos dar início à implementação do algoritmo KNN. O algoritmo funciona basicamente nas seguintes etapas:

- Calcular a distância de uma amostra para todas as demais;
- Pegar as n amostras mais próximas;
- Computar qual a classe mais frequente.

### Calculando a distância

In [ ]:
amostra_teste = teste[0,:4]
amostra_treino = treino[0,:4]

distancia = np.sum((amostra_treino - amostra_teste) ** 2) ** 0.5
print("Distância entre duas amostras quaisquer:", distancia)

Também é possível calcular a distância para todas as amostras de uma só vez.

In [ ]:
distancias = np.sum((amostra_teste - treino[:,:4]) ** 2, axis=1) ** 0.5
print("Distâncias de uma amostra de teste para todas as amostras de treino:")
print(distancias)

---

Em seguida, escolhemos quantos vizinhos vamos considerar e escolhemos os mais próximos da amostra avaliada.
A partir dos índices encontrados, calculamos qual é a classe mais frequente e atribuímos à amostra.

In [ ]:
k = 5
indices = distancias.argsort()[:k]
print("Indices dos vizinhos mais próximos:", indices)

In [ ]:
from collections import Counter

vizinhos = treino[indices,4]
print("Vizinhos mais próximos:", vizinhos)
classe = Counter(vizinhos).most_common(1)[0][0]
print("Classe mais frequente e quantidade:", Counter(vizinhos).most_common(1))

---

Para verificar se a classe encontrada pelo modelo está correta, basta comparar a saída com o rótulo da amostra.

In [ ]:
classe_certa = teste[0, 4]
print(classe_certa)
if classe_certa == classe:
    print("Correto")
else:
    print("Errado")

---

## Exercícios

### Utilizando as funções do scikit-learn para separar dados e treinar automaticamente o KNN, implemente as duas variações do Edit-KNN (para reduzir a base de dados). Em seguida, avalie o tamanho da base e o resultado de acurácia:

In [ ]:
# implemente seu código aqui

---

## Solução:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('iris.csv')
X = df.drop(columns=['classe'])
y = df.classe

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

FileNotFoundError: ignored

In [ ]:
# ELIMINACAO SEQUENCIAL

import warnings
warnings.filterwarnings("ignore")

import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# cria um vetor que vai auxiliar a identificar quais registros são considerados
# no treino e quais não são, como se fosse um filtro
indices = []
for _ in range(len(X_train)):
  indices.append(True)

# faz a logica de tentar eliminar cada registro e ver se está correto
for i in range(len(X_train)):
  indices[i] = False
  model = KNeighborsClassifier(n_neighbors = 1)
  model.fit(X_train[indices], y_train[indices])

  pred = model.predict([X_train.values[i]])
  if pred != y_train.values[i]:
    indices[i] = True

print("Quantas amostras de treino sobraram:", len(X_train[indices]))

# calcula o resultado final depois da redução

model = KNeighborsClassifier(n_neighbors = 1)
model.fit(X_train[indices], y_train[indices])
pred = model.predict(X_test)

print("Acurácia:", (pred == y_test).sum() / len(X_test) * 100)


Quantas amostras de treino sobraram: 9
Acurácia: 84.0


In [ ]:
# INSERCAO SEQUENCIAL

import warnings
warnings.filterwarnings("ignore")

import numpy as np
from sklearn.neighbors import KNeighborsClassifier

# cria um vetor que vai auxiliar a identificar quais registros são considerados
# no treino e quais não são, como se fosse um filtro
indices = []
for _ in range(len(X_train)):
  indices.append(False)

# inclui a primeira amostra ja que o modelo nunca vai acertar
indices[0] = True

# faz a logica de tentar inserir cada registro e ver se está correto
for i in range(1,len(X_train)):
  model = KNeighborsClassifier(n_neighbors = 1)
  model.fit(X_train[indices], y_train[indices])

  pred = model.predict([X_train.values[i]])
  if pred != y_train.values[i]:
    indices[i] = True

print("Quantas amostras de treino sobraram:", len(X_train[indices]))

# calcula o resultado final depois das insercoes

model = KNeighborsClassifier(n_neighbors = 1)
model.fit(X_train[indices], y_train[indices])
pred = model.predict(X_test)

print("Acurácia:", (pred == y_test).sum() / len(X_test) * 100)


Quantas amostras de treino sobraram: 14
Acurácia: 98.0
